# Setup

In [23]:
from PIL import Image, ImageFile, ImageDraw, ImageFont
import json
import time 
import timeit
import numpy as np

# what colors to use to display bounding boxes
# clases that are not in this dictionary will not be displayed
color_dict = {
    "Lung Opacity": '#ff6769',
    "Atelectasis": '#a87e56',
    "Consolidation": '#6bdd7b',
    "Fracture": '#82bde9',
    "Lung Lesion": '#eb69a5',
    "Pleural Effusion": '#8353a5',
    "Pleural Other": '#ffaa4d',
    "Tuberculosis": '#f4f156',
    "Pneumothorax": '#0392ce'
}

# the labels to be displayed next to each of the classes' bounding boxes
label_dict = {
    "BBOX_Atelectasis": "AT",
    "BBOX_Consolidation": "CO",
    "BBOX_Fracture": "FR",
    "BBOX_Lung Lesion": "LL",
    "BBOX_Lung Opacity": "LO",
    "BBOX_Pleural Effusion": "PE",
    "BBOX_Pleural Other": "PO",
    "BBOX_Pneumothorax": "PX",
    "BBOX_Tuberculosis": "TBC",
}

bboxes_width = 2 # width of bounding box sides
text_scale = 60 # size of the bounding box labels
benchmark_runs = 50 # how many times to run the drawing benchmark

# Benchmark

In [25]:
test_img_path = 'img.jpeg'
test_bbox_path = 'bounding_boxes.json'

img = Image.open(test_img_path).convert('RGB')
print(f'Loaded image {test_img_path}, shape: {img.size}')

# we scale the text according to the image dimensions and the scale
# the fnt object is used to draw text
text_size = img.height // text_scale * img.width // img.height
fnt = ImageFont.truetype("arial.ttf", int(text_size))

with open(test_bbox_path, 'r') as file:
    bbox_list = json.load(file)
print(f'Loaded {sum([len(l) for l in bbox_list])} BBoxes from {test_bbox_path}')
print(f'First BBox class {bbox_list[0]}')

benchmark_times = []
for benchmark_idx in range(benchmark_runs):
    # this is used to draw on the image
    # draw.rectangle and draw.text draw rectangles and text directly on the pixels of an image
    img_copy = img.copy()
    draw = ImageDraw.Draw(img_copy)

    a = timeit.default_timer()

    # go through each bbox_dict, each entry contains the class name and a list of rectangles representing the bounding boxes
    for bbox_dict in bbox_list:
        bbox_class = bbox_dict['type']

        # skip bounding boxes that are not meant to be displayed or that have no rectangles
        if bbox_class not in color_dict or len(bbox_dict['rectangles']) == 0:
            continue

        label_size = fnt.getsize(label_dict['BBOX_' + bbox_class])
        # label_size = (width, height) of the label text

        # go through each rectangle of the class
        for bbox in bbox_dict['rectangles']:
            draw.rectangle((bbox['start']['x'], bbox['start']['y'], bbox['end']['x'], bbox['end']['y']), outline=color_dict[bbox_class], width=bboxes_width)

            # MODIFY THIS
            # these are the coordinates at which we draw the label
            label_spot = bbox['start']['x'], bbox['start']['y'] - label_size[1] * 1.2
            draw.text(label_spot, label_dict['BBOX_' + bbox_class], font=fnt, fill=color_dict[bbox_class])

    b = timeit.default_timer()
    benchmark_time = b - a
    benchmark_times.append(benchmark_time)
    print(f'Benchmark {benchmark_idx} run time {b - a:.5f}s')
        
b_mean, b_std = np.mean(benchmark_times), np.std(benchmark_times)
print(f'Benchmark done, {benchmark_runs} runs, mean time {b_mean:.5f} (+/- {b_std:.5f})s')
        
img_copy.save('./out.jpg')
print('Saved image to out.jpg')

Loaded image img.jpeg, shape: (2332, 2733)
Loaded 68 BBoxes from bounding_boxes.json
First BBox class {'rectangles': [{'start': {'x': 1423.05225682259, 'y': 515.121554240584}, 'end': {'x': 1640.44490027428, 'y': 777.13093328476}}, {'start': {'x': 345.256063997746, 'y': 1078.28225076199}, 'end': {'x': 530.636279284954, 'y': 1248.74622729421}}, {'start': {'x': 520.591134786606, 'y': 554.086380198598}, 'end': {'x': 823.559723377228, 'y': 942.614909380674}}, {'start': {'x': 259.312173694372, 'y': 1207.61768880486}, 'end': {'x': 645.551268696785, 'y': 1791.77389025688}}, {'start': {'x': 587.193640112877, 'y': 969.645474046469}, 'end': {'x': 771.556459307671, 'y': 1188.39766618609}}, {'start': {'x': 1498.64119338989, 'y': 722.585422307253}, 'end': {'x': 1983.54347157478, 'y': 1439.55390715599}}, {'start': {'x': 347.860087871552, 'y': 874.160876691341}, 'end': {'x': 541.699132621288, 'y': 1144.51417145133}}, {'start': {'x': 1940.87566280365, 'y': 1701.15811443329}, 'end': {'x': 2138.249114751